In [ ]:
from lapy import TriaIO
from lapy import TetIO
from lapy import ShapeDNA

In [ ]:
tria = TriaIO.import_vtk("../data/cubeTria.vtk")
tet = TetIO.import_vtk("../data/cubeTetra.vtk")

In [ ]:
ShapeDNA.compute_shapedna(tria, k=3)

In [ ]:
ShapeDNA.compute_shapedna(tria, k=3, norm=True)

In [ ]:
ShapeDNA.compute_shapedna(tria, k=3, rwt=True)

In [ ]:
ShapeDNA.compute_shapedna(tet, k=3)

In [ ]:
ShapeDNA.compute_shapedna(tet, k=3, norm=True)

In [ ]:
ShapeDNA.compute_shapedna(tet, k=3, rwt=True)

In [ ]:
ShapeDNA.compute_asymmetry(tria, tria)

In [ ]:
ShapeDNA.compute_asymmetry(tria, tria, norm=True, rwt=True)

In [ ]:
ShapeDNA.compute_asymmetry(tet, tet)

In [ ]:
ShapeDNA.compute_asymmetry(tet, tet, norm=True, rwt=True)